<a id='summary'></a>
# Statistical matching
* Process taken from Horl2020
* matching attributes in Horl2020:
    * **age**
    * **gender**
    * **CS8**
    * **any car**
    * **income** (available for egt paris but not for egt lyon)
    * **departement**  (in our case, it's almost only 69 so no real point)
* minimum number of source observations : M=20
* Horl2020 adds income to census with filosofi, but for lyon income is not in egt, so we won't be able to join on it anyway.

* For each census row, we find the greatest N such that joining the census row with egt on the N _first_ matching attributes gives at least M matching egt rows. 
* Then, we select a row among the matching egt rows, weighted by their egt scaling weight (given by INSEE).

## Summary
* [Loading census](#census)
* [Loading EGT](#egt)
* [Statistical matching](#matching)
    * [Setting profiles for census](#profiles_census)
    * [Setting profiles for egt](#profiles_egt)
    * [Drawing egt rows for census](#draw)
* [Cleaning and exporting](#export)


In [1]:
import datetime
import pandas as pd
from tqdm import tqdm
import numpy as np
import json
import os

from utils.chrono import Chrono

with open('config.json', 'r') as config_path:
    config = json.load(config_path)

<a id='census'></a>
# Loading census
* [back to summary](#summary)

In [2]:
census_path = os.path.join(config['outdata_dir']['path'], config['outdata_dir']['synthpop_filename'])
census = pd.read_csv(census_path, dtype={'iris_or_commune':str})
print('{} persons in census'.format(len(census)))
census.head()

1325654 persons in census


,iris_or_commune,gender,has_car,occupation,age,home_status,main_transport_work
0,690270102,0,1,8,3,0,-1
1,690270102,0,1,8,3,0,-1
2,690270102,0,1,8,3,0,-1
3,690270102,1,1,8,3,0,-1
4,690270102,1,1,8,3,0,-1


## Setting `canton` and `commune`

In [3]:
cantville_commune = pd.read_excel('/Users/benoit/Desktop/Pro/210526-fusion/data/cantville_commune_2015/table-appartenance-geo-communes-15.xls',
                                 skiprows=5)
cantville_commune = cantville_commune.rename(columns={'CV':'canton'})
cantville_commune = cantville_commune[cantville_commune['CODGEO'].str[:2].isin(['69', '01', '38'])]

arr_lyon = ['{:.0f}'.format(69380+i) for i in range(1, 10)]
lyon = pd.DataFrame({'CODGEO':arr_lyon, 'canton':'69123'})
cantville_commune = pd.concat([cantville_commune, lyon], ignore_index=True)


census['commune'] = census['iris_or_commune'].str[:5]
census = census.merge(cantville_commune[['CODGEO','canton']], left_on='commune', right_on='CODGEO', how='left')


<a id='egt'></a>
# Loading EGT
* [back to summary](#summary)

In [4]:
egt_path = os.path.join(config['outdata_dir']['path'], config['outdata_dir']['HTS_filtered_filename'])
egt = pd.read_csv(egt_path, dtype={'zf_home':str})
egt = egt.groupby(['id_per']).first().reset_index()
print('{} persons in egt'.format(len(egt)))


activity_chain_path = os.path.join(config['outdata_dir']['path'], config['outdata_dir']['activity_chains_filename'])
activity_chains = pd.read_csv(activity_chain_path)
egt = egt[egt['id_per'].isin(activity_chains['id_per'])]
print('{} persons in egt with an activity chain'.format(len(egt)))
activity_chains.head()

18572 persons in egt
16941 persons in egt with an activity chain


,id_per,dep_0_motif,dep_0_zone,outsider,chain_len,zf_home,scaling_factor_person,dep_1_mode,dep_1_motif,dep_1_time,...,dep_10_motif,dep_10_time,dep_10_l2,dep_10_zone,dep_10_arrival_time,autonomy,work_tm_in_agenda,work_in_agenda,study_in_agenda,need_commute
0,1.010011e+09,0.0,101001.0,0.0,2.0,101001.0,54.14797,3.0,2.0,7.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,False,True,True
1,1.010018e+08,0.0,101001.0,0.0,2.0,101001.0,29.75996,3.0,4.0,10.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,False,False,False
2,1.010018e+08,0.0,101001.0,0.0,2.0,101001.0,51.85581,3.0,1.0,8.0,...,NaN,NaN,NaN,NaN,NaN,1.0,3.0,True,False,True
3,1.010018e+08,0.0,101001.0,0.0,2.0,101001.0,53.59816,3.0,2.0,7.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,False,True,True
4,1.010021e+09,0.0,101002.0,0.0,4.0,101002.0,50.09030,3.0,3.0,10.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,False,False,False


## Getting commune for each respondent of EGT
* EGT is defined on Zones Fines (ZF), incompatible with iris but still a subpartition of communes.
* Relying on the map of the "zone fine" we found to get the commune.

In [5]:
import geopandas as gpd
zf_path = os.path.join(config['outdata_dir']['path'], config['outdata_dir']['HTS_gis_filtered_filename'])
zf = gpd.read_file(zf_path)
zf['commune'] = zf['DComIris'].str[:5]
zf['commune'] = zf['commune'].replace({'A':'69290'})  # cleaning one zf

egt['zf_home'] = egt['zf_home'].str[2:]  # removing '00'
egt = egt.drop(columns=['commune'])
egt = egt.merge(zf[['ZF2015_Nouveau_codage', 'commune']], left_on='zf_home', right_on='ZF2015_Nouveau_codage', how='left')

print('{} out of {} ({:.0f}%) of egt respondant are outside of study area : commune set to None'.format(
    egt['commune'].isna().sum(), len(egt), 100*egt['commune'].isna().sum()/len(egt)))

egt = egt.merge(cantville_commune[['CODGEO','canton']], left_on='commune', right_on='CODGEO', how='left')

16941 out of 16941 (100%) of egt respondant are outside of study area : commune set to None


<a id='matching'></a>
# Statistical matching
* [back to summary](#summary)

In [6]:
# seuil de volume M
vol_thres = 20
matching_order = ['age', 'gender', 'occupation', 'has_car', 'home_status','canton' ]


<a id='profiles_census'></a>
## Setting profiles for census rows
* A profile consist in a number `imax` and a modality of the `imax` first variables in the `matching_order`. 
* Each census row is assigned a unique profile, with `imax` being the greatest `i` such that there are more than `vol_thres` EGT rows sharing the same values for the first `i` variables in the `matching_order`. 
* Note that an EGT row can be matched to different profiles, if there are profiles sharing the same first modalities but with a different `imax`.
* [back to summary](#summary)

In [7]:
census_grouped = (census
                  .groupby(matching_order)
                  .size().reset_index()
                  .rename(columns={0:'vol'}))

egt_grouped = (egt
               .groupby(matching_order)
               .size().reset_index()
               .rename(columns={0:'vol'}))

In [8]:
from IPython.display import display
print('census_grouped ({} joint modalities):'.format(len(census_grouped)))
display(census_grouped.head())
print('egt_grouped ({} joint modalities):'.format(len(egt_grouped)))
display(egt_grouped.head())

census_grouped (1457 joint modalities):


,age,gender,occupation,has_car,home_status,canton,vol
0,0,0,0,0,0,6906,3
1,0,0,0,0,0,69123,121
2,0,0,0,0,0,69ZZ,105
3,0,0,0,0,1,6905,2
4,0,0,0,0,1,6906,12


egt_grouped (0 joint modalities):


,age,gender,occupation,has_car,home_status,canton,vol


### Naive approach : what we get with a normal join

In [9]:
census_x_egt = census_grouped.merge(egt_grouped, on=matching_order, 
                                 how='left', suffixes=( '_census', '_egt'))
print('naive join on matching attributes :')

tmp = census_x_egt[census_x_egt['vol_egt']>=vol_thres]
print('{} groups ({:>4.1f}% of census rows) have >={} matching egt rows'.format(len(tmp), 
                                                100*tmp['vol_census'].sum()/len(census), vol_thres))

tmp = census_x_egt[(census_x_egt['vol_egt']<vol_thres)&(~census_x_egt['vol_egt'].isna())]
print('{} groups ({:>4.1f}% of census rows) have >0, <{} matching egt rows'.format(len(tmp),
                                                100*tmp['vol_census'].sum()/len(census), vol_thres))

tmp = census_x_egt[census_x_egt['vol_egt'].isna()]
print('{} groups ({:>4.1f}% of census rows) have no matching egt rows'.format(len(tmp),
                                                100*tmp['vol_census'].sum()/len(census), vol_thres))


naive join on matching attributes :
0 groups ( 0.0% of census rows) have >=20 matching egt rows
0 groups ( 0.0% of census rows) have >0, <20 matching egt rows
1457 groups (100.0% of census rows) have no matching egt rows


### Computing `imax` for each census group

In [10]:
mod_col = []
group_col = []
groupmatch_col = []
vol_col = []

census_grouped['imax']=0
for i in range(1, len(matching_order)+1):
    maskdf = egt_grouped.groupby(matching_order[:i])['vol'].sum().reset_index()

    census_grouped = census_grouped.merge(maskdf, on=matching_order[:i], suffixes=('','_mask'), how='left')
    census_grouped.loc[census_grouped['vol_mask']>=vol_thres, 'imax'] = i
    
    mod_col.append(len(census_grouped[matching_order[i-1]].unique()))
    group_col.append(len(maskdf))
    groupmatch_col.append((census_grouped.groupby(matching_order[:i])['vol_mask'].sum()>vol_thres).sum())
    vol_col.append('{:.0f}%'.format(100*((census_grouped['vol_mask']>vol_thres)*census_grouped['vol']).sum()/census_grouped['vol'].sum()))

    census_grouped.drop(columns=['vol_mask'], inplace=True)

report_df = pd.DataFrame({'var':matching_order,
                          '#mods':mod_col, 
                          '#groups':group_col, 
                          '#groups matching >={} egt rows'.format(vol_thres):groupmatch_col, 
                          '%vol':vol_col})

report_df

,var,#mods,#groups,#groups matching >=20 egt rows,%vol
0,age,4,0,0,0%
1,gender,2,0,0,0%
2,occupation,9,0,0,0%
3,has_car,2,0,0,0%
4,home_status,3,0,0,0%
5,canton,8,0,0,0%


#### Giving each census agent their `imax` 

In [11]:
len_before_merge = len(census)
census = census.merge(census_grouped.drop(columns=['vol']), on=matching_order)
assert len_before_merge == len(census), 'we lost rows by merging census with census_grouped'

#### Counting the number of groups when cropping after `imax` 

In [12]:
# replace vars after imax by None
census_grouped_censored = census_grouped.copy()
for nrow, row in tqdm(census_grouped_censored.iterrows(), total=len(census_grouped_censored)):
    row[matching_order[int(row['imax']):]] = None
    census_grouped_censored.iloc[nrow] = row
    
# groupby keeping None
census_grouped_censored = (census_grouped_censored
                           .groupby(['imax']+matching_order, dropna=False)
                           ['vol'].sum()
                           .reset_index())

tqdm.write('when considering only the imax first vars, getting {} different profiles for statistical matching'.format(len(census_grouped_censored)))

100%|█████████████████████████████████████| 1457/1457 [00:01<00:00, 1188.17it/s]

when considering only the imax first vars, getting 1 different profiles for statistical matching


<a id='profiles_egt'></a>
## Finding EGT rows for each profile
* For each profile in census, we create a dataframe of all EGT rows matching the profile.
* Recall that an EGT row can be matched to differents profiles, if there are profiles sharing the same first modalities but with a different `imax`.
* [back to summary](#summary)

In [13]:
def get_mask(df, target_row, matching_order, imax):
    """
    return a mask column of df rows that match target_row up to the imax first columns in matching_order
    """
    target_arr = target_row[matching_order].values[:imax]
    mask = (df[matching_order].values[:, :imax] == target_arr).all(axis=1)
    return mask

def get_fold(df, target_row, matching_order, imax):
    """
    return a dataframe made of rows of df that match target_row up to the imax first columns in matching_order
    """
    mask = get_mask(df, target_row, matching_order, int(imax))
    return df.loc[df[mask].index, :]


### EGT folded
* For each join key in census, getting the sub-df of EGT containing all the rows of EGT matching this key.
* We use it after to perform the join.
* So this amounts to a merge with variable join key


In [14]:
egt_folded = {}
for k, row in tqdm(census_grouped_censored.iterrows(), total=len(census_grouped_censored)):
    key = tuple(row[matching_order[:int(row['imax'])]].values)
    fold = get_fold(egt, row, matching_order, row['imax'])
    # Normalising weights of rows inside each fold
    fold.loc[fold.index, 'weight'] = fold['scaling_factor_person']/fold['scaling_factor_person'].sum()
    egt_folded[key] = fold



100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 32.96it/s]


#### Sanity check

In [15]:
# Checking weights of rows
for k,fold in egt_folded.items():
    if len(fold)<vol_thres:
        print('error: found a group with not enough egt individuals :' ,k, len(fold))

# just checking that nothing went wrong when attributing values to DFs that were taken from another DF
check = True
for k,fold in egt_folded.items():
    check = check and np.all(egt_folded[k].loc[egt_folded[k].index, 'weight'] == egt_folded[k]['scaling_factor_person']/egt_folded[k]['scaling_factor_person'].sum())
if check:
    print('everything ok, no error detected that could be due to pandas whims')
else:
    print('something went wrong when attributing values to columns in df that were copies of a df')


everything ok, no error detected that could be due to pandas whims


<a id='draw'></a>
## Drawing an agenda for each row in the census
* Each census row has a profile, and each profile has a list of EGT rows. 
* For each profile matching $C$ census rows and $E$ egt rows, we draw $C$ egt rows with replacement and assign their agendas to the census rows.
* Drawing of egt rows is made with weights defined by the scaling factors of the egt rows
* [back to summary](#summary)

In [16]:
# 00:02:00

census['id_agenda'] = -1
for k, row in tqdm(census_grouped_censored.iterrows(), total=len(census_grouped_censored)):
    imax = int(row['imax'])
    t=tuple(row[matching_order[:imax]].values)
    
    census_mask = get_mask(census, row, ['imax']+matching_order, imax+1)  # the imax must match too
    
    agenda_draw = np.random.choice(egt_folded[t]['id_per'].values, 
                                   int(census_mask.sum()),
                                   p=egt_folded[t]['weight'])
    
    census.loc[census[census_mask==1].index, 'id_agenda'] = agenda_draw


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.10it/s]


<a id='export'></a>
# Cleaning & Exporting
* [back to summary](#summary)

In [17]:
out_path = os.path.join(config['outdata_dir']['path'], config['outdata_dir']['synthpop_statmatch_filename'])
print(datetime.datetime.now())
print('Exported to:')
print(out_path)
census = census.drop(columns=['commune', 'CODGEO', 'canton', 'imax'])
census.to_csv(out_path, index=False)
census.head()

2023-11-28 06:48:55.169272
Exported to:
/Users/benoit/Desktop/Pro/210526-fusion/outdata/synthpop/synthpop_statmatch.csv


,iris_or_commune,gender,has_car,occupation,age,home_status,main_transport_work,id_agenda
0,690270102,0,1,8,3,0,-1,1320021221
1,690270102,0,1,8,3,0,-1,10800131
2,690270102,0,1,8,3,0,-1,243003762
3,690270102,0,1,8,3,0,-1,229755801
4,690270102,0,1,8,3,0,-1,7050031782
